# NLP - Lab 02
## Adrien Giget, Tanguy Malandain, Denis Stojiljkovic

# Imports

In [ ]:
import datasets as ds
import numpy as np
from typing import Dict, Any, TypeVar
import re
from collections import Counter

In [2]:
data = ds.load_dataset("imdb")

data

Found cached dataset imdb (C:/Users/Tanguy/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
100%|██████████| 3/3 [00:00<00:00, 33.70it/s]


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

## The dataset (3 points)

   * How many splits does the dataset has? (1 point)
   * How big are these splits? (1 point)
   * What is the proportion of each class on the supervised splits? (1 point)

In [3]:
len(data)

3

On a 3 sets de données.

In [4]:
data.num_rows

{'train': 25000, 'test': 25000, 'unsupervised': 50000}

Le set "train" à 25 000 données, ainsi que le set "test".
Le set "unsupervised" en a 50 000.

On enlève le dataset unsupervised qui ne sera plus utile.

In [5]:
del data["unsupervised"]

In [6]:
nb_positive = data.filter(lambda x: x["label"] == 1)
nb_positive

Loading cached processed dataset at C:\Users\Tanguy\.cache\huggingface\datasets\imdb\plain_text\1.0.0\d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0\cache-00a580d4a08599a8.arrow
Loading cached processed dataset at C:\Users\Tanguy\.cache\huggingface\datasets\imdb\plain_text\1.0.0\d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0\cache-6fdcb91b1f3cc61f.arrow


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 12500
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 12500
    })
})

In [7]:
nb_negative = data.filter(lambda x: x["label"] == 0)
nb_negative

Loading cached processed dataset at C:\Users\Tanguy\.cache\huggingface\datasets\imdb\plain_text\1.0.0\d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0\cache-c3579795bcf1a351.arrow
Loading cached processed dataset at C:\Users\Tanguy\.cache\huggingface\datasets\imdb\plain_text\1.0.0\d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0\cache-20cc5fc08326d82a.arrow


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 12500
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 12500
    })
})

Dans les deux sets supervisés, nous avons la moitié des données labellisée positive et l'autre négative, donc 12500 de chaque.

## Naive Bayes classifier (13 points)

* (2 points) Take a look at the data and create an adapted preprocessing function with at least:

  * Lower case the text.
  * Replace punctuations with spaces (you can use from string import punctuation to ease your work). Think that maybe not all punctuations should be removed or replaced.


In [ ]:
def get_lower(comment: Dict[str, Any]) -> Dict[str, Any]:
    """
    :param comment: Actual customer comment as a dict { str : any }
    :return: a modified dict { str : any }
    """
    comment["text"] = comment["text"].lower()
    return comment

adapted_data = data.map(get_lower)

In [9]:
adapted_data["train"]["text"][8]

'who are these "they"- the actors? the filmmakers? certainly couldn\'t be the audience- this is among the most air-puffed productions in existence. it\'s the kind of movie that looks like it was a lot of fun to shoot\x97 too much fun, nobody is getting any actual work done, and that almost always makes for a movie that\'s no fun to watch.<br /><br />ritter dons glasses so as to hammer home his character\'s status as a sort of doppleganger of the bespectacled bogdanovich; the scenes with the breezy ms. stratten are sweet, but have an embarrassing, look-guys-i\'m-dating-the-prom-queen feel to them. ben gazzara sports his usual cat\'s-got-canary grin in a futile attempt to elevate the meager plot, which requires him to pursue audrey hepburn with all the interest of a narcoleptic at an insomnia clinic. in the meantime, the budding couple\'s respective children (nepotism alert: bogdanovich\'s daughters) spew cute and pick up some fairly disturbing pointers on \'love\' while observing their 

In [ ]:
def remove_angle_brackets(comment: Dict[str, Any]) -> Dict[str, Any]:
    """
    Remove everything inside < > characters (usually <br > from scraping)
    :param comment: Actual customer comment as a dict { str : any }
    :return: a modified dict { str : any }
    """
    comment["text"] = re.sub(f'<.*?>', '', comment["text"])
    return comment

def remove_unknown_unicode(comment: Dict[str, Any]) -> Dict[str, Any]:
    """
    Remove all non-usual unicode characters (as —)
    :param comment: Actual customer comment as a dict { str : any }
    :return: a modified dict { str : any }
    """
    comment["text"] = re.sub(r"(\\x\S+)", '', repr(comment["text"])[1:-1])
    return comment

def remove_backslash(comment: Dict[str, Any]) -> Dict[str, Any]:
    """
    Remove all backslash (usually as "it\'s")
    :param comment: Actual customer comment as a dict { str : any }
    :return: a modified dict { str : any }
    """
    comment["text"] = re.sub(r"\\", '', comment["text"])
    return comment

def remove_ponctuation(comment: Dict[str, Any]) -> Dict[str, Any]:
    """
    Remove non-wanted ponctuations as _\(\)\".,`:;* that don't add much syntactical meaning.
    :param comment: Actual customer comment as a dict { str : any }
    :return: a modified dict { str : any }
    """
    comment["text"] = re.sub(r"[_\(\)\".,`:;*]", ' ', comment["text"])
    comment["text"] = re.sub(r"[(!?)]", r" \g<0> ", comment["text"]) # We want to keep ? and ! in the split
    return comment

def remove_all(comment: Dict[str, Any]) -> Dict[str, Any]:
    """
    Apply all above
    :param comment: Actual customer comment as a dict { str : any }
    :return: a modified dict { str : any }
    """
    return remove_ponctuation(remove_backslash(remove_unknown_unicode(remove_angle_brackets(get_lower(comment)))))

adapted_data = data.map(remove_all)

In [11]:
adapted_data["train"]["text"][8]

"who are these  they - the actors ?  the filmmakers ?  certainly couldn't be the audience- this is among the most air-puffed productions in existence  it's the kind of movie that looks like it was a lot of fun to shoot too much fun  nobody is getting any actual work done  and that almost always makes for a movie that's no fun to watch ritter dons glasses so as to hammer home his character's status as a sort of doppleganger of the bespectacled bogdanovich  the scenes with the breezy ms  stratten are sweet  but have an embarrassing  look-guys-i'm-dating-the-prom-queen feel to them  ben gazzara sports his usual cat's-got-canary grin in a futile attempt to elevate the meager plot  which requires him to pursue audrey hepburn with all the interest of a narcoleptic at an insomnia clinic  in the meantime  the budding couple's respective children  nepotism alert  bogdanovich's daughters  spew cute and pick up some fairly disturbing pointers on 'love' while observing their parents   ms  hepburn 

* (4 points) Implement your own naive Bayes classifier from scratch. The pseudocode can be found in the slides or the book reference.

In [12]:
CustomClass = TypeVar('CustomClass')

class CustomNaivesBayesClassifier:
    """
    Our custom Naive Bayes Classifier from Speech and Language Processing (3rd ed. draft)
    by Dan Jurafsky and James H. Martin
    """
    def __init__(self) -> None:
        self._logPrior: Dict[CustomClass, float] = {}
        self._logLikelihoods: Dict[str, Dict[CustomClass, float]] = {}
        self._classes: list[CustomClass] = []
        self.__vocabulary: set[str] = set()


    def train_naive_bayes(self, data: list[str], label: list[CustomClass]) -> None:
        """
        Implementation of the training algorithm
        :param data: All comments made by customers
        :param label: The associated label
        :return: None
        """
        n_doc = len(data)
        self._classes = np.unique(label)
        self.__vocabulary =  set(word for text in data for word in text.split())
        self._logLikelihoods = {word: {} for word in self.__vocabulary}

        for classe in self._classes:
            n_c = len([index for index in range(len(data)) if label[index] == classe])
            self._logPrior[classe] = np.log(n_c / n_doc)
            big_doc =  [word for index in range(len(data)) if label[index] == classe for word in data[index].split()]
            denominator = len(big_doc) + len(self.__vocabulary)
            counter = Counter(big_doc)

            for word in self.__vocabulary:
                self._logLikelihoods[word][classe] = np.log((counter[word] + 1) / denominator)


    def test_naive_bayes(self, data: list[str]) -> list[CustomClass]:
        """
        The test of train_naive_bayes()
        :param data: Testing non labeled data
        :return: A list of class
        """
        res = []

        for text in data:
            sum_c = self._logPrior.copy()
            for word in text.split():
                if word in self.__vocabulary:
                    for classe in self._classes:
                        sum_c[classe] += self._logLikelihoods[word][classe]
            res.append(max(sum_c, key=sum_c.get))

        return res
    
    def get_10_highest_log(self) -> Dict[CustomClass, list[str]]:
        """
        Get the 10 highest likehood in each class
        :return: A dict of class containing list of str
        """
        res = {}
        for classe in self._classes:
            logprior_c = {key: value[classe] for key, value in self._logLikelihoods.items()}
            res[classe] = sorted(logprior_c, key=logprior_c.get, reverse=True)[:10]
        
        return res


In [13]:
custom = CustomNaivesBayesClassifier()
custom.train_naive_bayes(adapted_data["train"]["text"], adapted_data["train"]["label"])

In [14]:
custom_bayes_results = custom.test_naive_bayes(adapted_data["test"]["text"])

* (3 points) Implement a naive Bayes classifier using scikit-learn.

    * Use a scikit-learn Pipeline with a CountVectorizer and MultinomialNB classifier. You can use other


In [15]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB())
])

pipeline.fit(adapted_data["train"]["text"], adapted_data["train"]["label"]);

In [16]:
sklearn_results = pipeline.predict(adapted_data["test"]["text"])

* (1 point) Report the accuracy on both training and test set, for both your implementation and the scikit-learn one.

In [17]:
def accuracy_bayes(predicted_data: list[Any], real_data: list[Any]) -> float:
    return len([index for index in range(len(predicted_data)) if real_data[index] == predicted_data[index]]) / len(predicted_data)

In [18]:
custom_train_data = CustomNaivesBayesClassifier()
custom_train_data.train_naive_bayes(adapted_data["train"]["text"], adapted_data["train"]["label"])
custom_train_data_results = custom_train_data.test_naive_bayes(adapted_data["train"]["text"])
accuracy_bayes(custom_train_data_results, adapted_data["train"]["label"])

0.9078

In [19]:
pipeline_train_data = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB())
])

pipeline_train_data.fit(adapted_data["train"]["text"], adapted_data["train"]["label"])
sklearn_train_data = pipeline.predict(adapted_data["train"]["text"])
accuracy_bayes(sklearn_train_data, adapted_data["train"]["label"])

0.898

In [20]:
data_test_label = adapted_data["test"]["label"]

accuracy_bayes(custom_bayes_results, data_test_label)

0.81592

In [21]:
accuracy_bayes(sklearn_results, data_test_label)

0.81432

* (1 point) Most likely, the scikit-learn implementation will give better results. Looking at the documentation, explain why it could be the case.

In [22]:
%%time

custom.train_naive_bayes(adapted_data["train"]["text"], adapted_data["train"]["label"])
custom.test_naive_bayes(adapted_data["test"]["text"]);

CPU times: total: 8.33 s
Wall time: 8.68 s


In [23]:
%%time

pipeline.fit(adapted_data["train"]["text"], adapted_data["train"]["label"])
pipeline.predict(adapted_data["test"]["text"]);

CPU times: total: 6.16 s
Wall time: 6.32 s


D'après la documentation, la bibliothèque scikit-learn utilise quelques méthodes en plus d'un simple algorithme de Bayes naïf.
Notamment un preprocessing des données : tokénisation du text et suppression des "stops words".
Et une vectorisation du corpus de mots - en transformant donc chaque mot par un nombre - pour faciliter les calculs.


Mais bien que plus lente, on remarque que notre version est plus performante en termes de précision de ses prédictions.

* (1 point) Why is accuracy a sufficient measure of evaluation here?

La précision de prédiction est une bonne mesure d'évaluation dans notre cas notre set de données de test est déjà labellisé.
Nous sommes donc certains de l'efficacité de l'implémentation avec la seule mesure de la précision.

* (1 point) Using one of the implementation, take at least 2 wrongly classified example from the test set and try explaining why the model failed.

In [24]:
wrong_predicted_data = [data["test"][index] for index in range(len(sklearn_results)) if data_test_label[index] != sklearn_results[index]]
len(wrong_predicted_data)

4642

In [25]:
# instantiate a Pseudo-random number generator (PRNG)
rng = np.random.default_rng(420)

wrong_predicted_data[int(rng.random() * len(wrong_predicted_data))]

{'text': "Ray is interesting in parts, and technically it's very well made, but Ray is often sluggish, and forgets some important details about Ray's life. All the movie shows us is parts where he's in his prime, and most successful, which is good, it's just I wanted to see some bits about his older life too. Jamie Fox mimics Ray Charles to a t, at times it's absolutely uncanny. I have to say, Jamie is the reason I got through this movie. The 1st half is a lot better then the 2nd. It's more interesting, it has more Oomph, and it's nowhere near as sluggish as the 2nd. I wasn't a big fan of Ray Charles to begin with to be honest, so I really didn't have any expectations for the film, what so ever. Ray's biggest problem had to be the over length. This could've been cut very easily, with more relevant scenes, other then the ones they used. I found the early part of Ray's life when he was just starting to get successful, the most interesting. He was humble back then, and somewhat a gentlema

On remarque que ce commentaire est une critique constructive du film.
Par exemple dans la phrase : "Ray is interesting at times, dreadfully dull at others. When all was said and done, I was disappointed by how routine it seemed at times. Just because it's a bibliographical film, Doesen't mean it's automatically Oscar worthy.", l'individu n'hésite pas à pointer du doigt ce qui ne lui a pas plu bien qu'appréciant le film.
Il est donc compliqué de se baser sur une simple analyse syntaxique pour déterminer si c'est une revue positive ou négative quand l'individu est particulièrement critique de certains points.

In [26]:
wrong_predicted_data[int(rng.random() * len(wrong_predicted_data))]

{'text': 'I have to hold Barney drilling my head every day; well.. I guess there must be reasons. First, I\'m convinced that our kids are not stupids, they are just kids, but they know (my 1 and a half years old son "selects" what to see) what\'s nice or disgusting. Did you see the news? Do you think your kids HAVE TO KNOW the reality as it is? Maybe..or maybe not; we (the adults) have the responsibility about what we want for our kids, and what to teach them. A film of drug dealers? news about massacres in Middle East? Of course, the kids must know there is a Real Life, but... they are kids; let\'s give them some mercy. What do you want for them? If you wanna have kids trained on weapons or the best way to kill a neighbor, go ahead, impose them Lethal Weapon, Kill Bill, any manga\'s anime, tell them Santa\'s a depraved who enters through the chimney directly to violate them. I want illusions for my son (don\'t get me wrong, I\'m not saying Barney and Friends is the best; in fact, the 

On remarque dans cette critique un lexique de mots connotés négativement : "disgusting", "drug dealers", "fear", "lethal" ou encore "violate".
De plus, cette critique est plus un avis sur ce que l'auteur pense de la société. Et même s'il a pu aimer le film, il n'en parle jamais directement. Il ne subsiste donc que très peu de mots ayant un registre positif ("happy" à la fin) pour une critique qui l'est.

Nous pouvons ajouter que des tournures de phrases peu conventionnelles ou l'utilisation de l'ironie induit fortement en erreur ce type d'analyse.

* [Bonus] What are the top 10 most important words (features) for each class? (bonus points)

    * Look at the words with the highest likelihood in each class (if you use scikit-learn, you want to check feature_log_prob_).
    * Remove stopwords (see NLTK stopwords corpus) and check again.



### Les mots les plus significatifs avec les stop words

In [27]:
custom.get_10_highest_log()

{0: ['the', 'a', 'and', 'of', 'to', 'is', 'in', 'this', 'i', 'it'],
 1: ['the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i', 'this']}

Les mots les plus significatifs correspondent à des stop words, ce sont des mots qui n'aident pas à trouver à quelle classe appartient le texte.

### Pré-processing des "stop words"

In [ ]:
import nltk
nltk.download('stopwords')

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

stops = set(stopwords.words('english'))

In [29]:
def remove_stop_words(comment: Dict[str, Any], stop_words: set = stops) -> Dict[str, Any]:
    """
    Remove every english stop words as defined in the nltk package
    :param comment: Actual customer comment as a dict { str : Any }
    :param stopWords: A set of stop words
    :return: a modified dict { str : Any }
    """
    words = word_tokenize(comment["text"])
    words_filtered = [w for w in words if w not in stop_words]
    comment["text"] = ' '.join(words_filtered)
    return comment

In [30]:
adapted_data_no_stop = adapted_data.map(remove_stop_words)

In [31]:
adapted_data_no_stop["train"]["text"][8]

"- actors ? filmmakers ? certainly could n't audience- among air-puffed productions existence 's kind movie looks like lot fun shoot much fun nobody getting actual work done almost always makes movie 's fun watch ritter dons glasses hammer home character 's status sort doppleganger bespectacled bogdanovich scenes breezy ms stratten sweet embarrassing look-guys-i'm-dating-the-prom-queen feel ben gazzara sports usual cat's-got-canary grin futile attempt elevate meager plot requires pursue audrey hepburn interest narcoleptic insomnia clinic meantime budding couple 's respective children nepotism alert bogdanovich 's daughters spew cute pick fairly disturbing pointers 'love ' observing parents ms hepburn drawing dignity manages rise proceedings- monumental challenge playing ostensibly everybody looks great ? 's movie expect much 's 're looking 'd better picking copy vogue oh- mentioned colleen camp thoroughly annoys even apart singing competent wholly unconvincing country western numbers w

### Testing this corpus with our Naives Bayes predictor

In [32]:
custom_2 = CustomNaivesBayesClassifier()
custom_2.train_naive_bayes(adapted_data_no_stop["train"]["text"], adapted_data_no_stop["train"]["label"])

In [33]:
custom_bayes_results_no_stop = custom_2.test_naive_bayes(adapted_data_no_stop["test"]["text"])

In [34]:
accuracy_bayes(custom_bayes_results_no_stop, data_test_label)

0.82776

Ce pré-processing des données permet bel est bien d'améliorer la qualité de prédiction de l'algorithme. Nous passons d'une prédiction correcte à 81.536% à 82.776%

Donc un gain absolu de 1.24% et un gain relatif de 1.07%.

### Testing with sky-learn

In [35]:
pipeline_no_stop = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB())
])

pipeline_no_stop.fit(adapted_data_no_stop["train"]["text"], adapted_data_no_stop["train"]["label"]);

In [36]:
sklearn_bayes_no_stop = pipeline_no_stop.predict(adapted_data_no_stop["test"]["text"])

In [37]:
accuracy_bayes(sklearn_bayes_no_stop, data_test_label)

0.82468

Encore une fois, nous améliorons la qualité de prédiction, on passe de 81.432% de bonne prédiction à 82.468% avec ce pré-processing.

Un gain absolu de 1.036% et relatif de 1.06%.

Cette différence doit être due au fait que sky-learn traite déjà les données pour réduire ce problème.

* [Bonus] Play with scikit-learn's version parameters. For example, see if you can consider unigram and bigram instead of only unigrams.

In [38]:
pipeline_bigram = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2))),
    ('clf', MultinomialNB())
])

pipeline_bigram.fit(adapted_data_no_stop["train"]["text"], adapted_data_no_stop["train"]["label"]);

In [39]:
sklearn_bayes_bigram = pipeline_bigram.predict(adapted_data_no_stop["test"]["text"])

In [40]:
accuracy_bayes(sklearn_bayes_bigram, data_test_label)

0.85276

L'utilisation de bigram améliore la qualité de prédiction, nous passons de 81.432% à 85.276% ce qui est une amélioration absolue de 3.844% et relative de 26%.

In [41]:
# Use of unigram in skikit-learning
pipeline_unigram = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 1))),
    ('clf', MultinomialNB())
])

pipeline_unigram.fit(adapted_data_no_stop["train"]["text"], adapted_data_no_stop["train"]["label"]);

In [42]:
sklearn_bayes_unigram = pipeline_unigram.predict(adapted_data_no_stop["test"]["text"])

In [43]:
accuracy_bayes(sklearn_bayes_unigram, data_test_label)

0.82468

L'utilisation d'unigram améliore également la qualité de prédiction, nous passons de 81.432% à 82.468% ce qui est une amélioration absolue de 0.936% et relative de 1.15%.

La différence est cependant moins prononcée.

### Les mots les plus significatifs sans les stop words

In [44]:
custom_2.get_10_highest_log()

{0: ["'s", 'movie', "n't", 'film', 'one', '!', '?', 'like', 'would', 'even'],
 1: ["'s",
  'film',
  'movie',
  "n't",
  'one',
  '!',
  'like',
  'good',
  'story',
  'great']}

Cette fois-ci certains mots sont différents, dans le label positif on retrouve donc des mots élogieux comme 'good' ou 'great'.
Cependant, pour le label négatif, à part le fait que "n't" soit en 3ᵉ position, nous ne retrouvons pas de mots connotés négativement dans le langage courant.

# Stemming and Lemmatization (4 points)

* (2 points) Add stemming or lemmatization to your pretreatment.

On va effectuer une stemmisation grâce à la bibliothèque NLTK.

In [45]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [46]:
def stemming(comment: Dict[str, Any], stemmer: PorterStemmer = ps) -> Dict[str, Any]:
    """
    Modify every word with the stemming implementation
    :param [IN] stemmer:
    :param comment: Actual customer comment as a dict { str : Any }
    :return: a modified dict { str : Any }
    """
    words = word_tokenize(comment["text"])
    words_stemmed = [stemmer.stem(w) for w in words]
    comment["text"] = ' '.join(words_stemmed)
    return comment

* (1 point) Train and evaluate your model again with these pretreatment.

In [47]:
adapted_data_stemmed = adapted_data_no_stop.map(stemming)

# Test de la stemmisation sur notre implémentation

In [48]:
custom_3 = CustomNaivesBayesClassifier()
custom_3.train_naive_bayes(adapted_data_stemmed["train"]["text"], adapted_data_stemmed["train"]["label"])

In [49]:
custom_bayes_results_stemming = custom_3.test_naive_bayes(adapted_data_stemmed["test"]["text"])
len(custom_bayes_results_stemming)

25000

In [50]:
accuracy_bayes(custom_bayes_results_stemming, data_test_label)

0.81956

* (1 point) Are the results better or worse? Try explaining why the accuracy changed.

Nous obtenons 0.81956% de réussite, contre une précision de 0.82776% précédemment.

Nous pouvons expliquer ce résultat par le fait que la langue anglaise n'est pas une langue qui se construit sur la suffixation de racines contrairement à des langues comme l'allemand ou le turc.

Ainsi, réduire le lexique pourrait mener à perdre en précision quant aux sens des différents mots.

# Test de la stemmisation sur sk-learn

In [51]:
pipeline_stem = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB())
])

pipeline_stem.fit(adapted_data_stemmed["train"]["text"], adapted_data_stemmed["train"]["label"]);

In [52]:
sklearn_bayes_stem = pipeline_stem.predict(adapted_data_stemmed["test"]["text"])

In [53]:
accuracy_bayes(sklearn_bayes_stem, data_test_label)

0.81744

Nous obtenons 0.81744% de réussite, contre une précision de 0.82468% précédemment.

Même commentaire sur l'impact de la stemmisation.